# Customer Analysis - Model Customer Behavior


## Import

In [1]:
import os
import pyspark
import pandas as pd
import pyspark.sql.functions as f
import pyspark.sql.types as T
import plotly.express as px
import plotly.graph_objects as go

from pyspark.mllib.clustering import KMeans
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.util import MLUtils

## Read

In [2]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("app_great") \
    .config("spark.executor.memory", f"16g") \
    .config("spark.driver.memory", f"16g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", f"16g") \
    .config("spark.sql.debug.maxToStringFields", f"16") \
    .getOrCreate()

In [3]:
# # Testing / Debug

sdf = spark.read.csv("data/test_data.csv", header=True, inferSchema=True)


In [4]:

# sdf_201911 = spark.read.csv("data/2019-Nov.csv", header=True, inferSchema=True)
# sdf_201910 = spark.read.csv("data/2019-Oct.csv", header=True, inferSchema=True)

# sdf = sdf_201910.union(sdf_201911)
# sdf = sdf_201911

# sdf.show(5)

## Preparation

In [5]:
# Datatypes
sdf = sdf.withColumn("event_time", sdf["event_time"].cast(pyspark.sql.types.TimestampType()))
sdf = sdf.withColumn("category_id", sdf["category_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("product_id", sdf["product_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("user_id", sdf["user_id"].cast(pyspark.sql.types.StringType()))

# Feature Splitting
sdf = sdf.withColumn("category_class", f.split(sdf["category_code"], "\.").getItem(0))
sdf = sdf.withColumn("category_sub_class", f.split(sdf["category_code"], "\.").getItem(1))
sdf = sdf.withColumn("category_sub_sub_class", f.split(sdf["category_code"], "\.").getItem(2))

sdf = sdf.withColumn("year", f.year("event_time"))
sdf = sdf.withColumn("month", f.month("event_time"))
sdf = sdf.withColumn("weekofyear", f.weekofyear("event_time"))
sdf = sdf.withColumn("dayofyear", f.dayofyear("event_time"))
sdf = sdf.withColumn("dayofweek", f.dayofweek("event_time"))
sdf = sdf.withColumn("dayofmonth", f.dayofmonth("event_time"))
sdf = sdf.withColumn("hour", f.hour("event_time"))

# None Handling
# sdf = sdf.fillna(value="not defined")

sdf.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_class: string (nullable = true)
 |-- category_sub_class: string (nullable = true)
 |-- category_sub_sub_class: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- dayofyear: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- hour: integer (nullable = true)



## Data Creation

In [6]:
sdf_session = sdf.select("user_id", "user_session", "event_type", "product_id", "price", "event_time")

sdf_session = sdf_session.withColumn("bought_product", f.when(sdf_session.event_type == "purchase", sdf_session["product_id"]).otherwise(None))

sdf_session = sdf_session.withColumn("views", f.when(sdf_session.event_type == "view", 1).otherwise(0))
sdf_session = sdf_session.withColumn("purchases", f.when(sdf_session.event_type == "purchase", 1).otherwise(0))
sdf_session = sdf_session.withColumn("carts", f.when(sdf_session.event_type == "cart", 1).otherwise(0))

sdf_session = sdf_session.withColumn("first_event", sdf_session.event_time)
sdf_session = sdf_session.withColumn("last_event", sdf_session.event_time)

In [7]:
sdf_session.show()

+---------+--------------------+----------+----------+------+-------------------+--------------+-----+---------+-----+-------------------+-------------------+
|  user_id|        user_session|event_type|product_id| price|         event_time|bought_product|views|purchases|carts|        first_event|         last_event|
+---------+--------------------+----------+----------+------+-------------------+--------------+-----+---------+-----+-------------------+-------------------+
|520088904|4d3b30da-a5e4-49d...|      view|   1003461|489.07|2019-11-01 01:00:00|          null|    1|        0|    0|2019-11-01 01:00:00|2019-11-01 01:00:00|
|530496790|8e5f4f83-366c-4f7...|      view|   5000088|293.65|2019-11-01 01:00:00|          null|    1|        0|    0|2019-11-01 01:00:00|2019-11-01 01:00:00|
|561587266|755422e7-9040-477...|      view|  17302664| 28.31|2019-11-01 01:00:01|          null|    1|        0|    0|2019-11-01 01:00:01|2019-11-01 01:00:01|
|518085591|3bfb58cd-7892-48c...|      view|   

In [8]:
sdf_session_agg = sdf_session.groupBy("user_id", "user_session").agg(f.avg("price"), f.sum("views"), f.sum("purchases"), f.sum("carts"), f.min("event_time"), f.max("event_time"), f.collect_list("bought_product"))
sdf_session_agg = sdf_session_agg.withColumn("duration", (sdf_session_agg["max(event_time)"] - sdf_session_agg["min(event_time)"]))
sdf_session_agg = sdf_session_agg.withColumn("sum(events)", (sdf_session_agg["sum(views)"] + sdf_session_agg["sum(purchases)"] + sdf_session_agg["sum(carts)"]))
sdf_session_agg = sdf_session_agg.withColumn("turnover", f.when(sdf_session_agg["sum(purchases)"] > 0, (sdf_session_agg["sum(purchases)"] *  sdf_session_agg["avg(price)"])).otherwise(0))

sdf_session_agg = sdf_session_agg.withColumn("successfull", f.when(sdf_session_agg["sum(purchases)"] > 0, 1).otherwise(0))
sdf_session_agg.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)
 |-- avg(price): double (nullable = true)
 |-- sum(views): long (nullable = true)
 |-- sum(purchases): long (nullable = true)
 |-- sum(carts): long (nullable = true)
 |-- min(event_time): timestamp (nullable = true)
 |-- max(event_time): timestamp (nullable = true)
 |-- collect_list(bought_product): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- duration: interval (nullable = true)
 |-- sum(events): long (nullable = true)
 |-- turnover: double (nullable = true)
 |-- successfull: integer (nullable = false)



In [9]:
# sdf_session_agg.show()

In [10]:
sdf_session_agg.where(sdf_session_agg["turnover"] > 0).show()

+---------+--------------------+----------+----------+--------------+----------+-------------------+-------------------+----------------------------+----------+-----------+--------+-----------+
|  user_id|        user_session|avg(price)|sum(views)|sum(purchases)|sum(carts)|    min(event_time)|    max(event_time)|collect_list(bought_product)|  duration|sum(events)|turnover|successfull|
+---------+--------------------+----------+----------+--------------+----------+-------------------+-------------------+----------------------------+----------+-----------+--------+-----------+
|559368633|d6034fa2-41fb-4ac...|     566.3|         2|             1|         0|2019-11-01 01:00:41|2019-11-01 01:01:20|                  [13200605]|39 seconds|          3|   566.3|          1|
|513351129|e6b7ce9b-1938-4e2...|    211.92|         1|             1|         0|2019-11-01 01:00:17|2019-11-01 01:01:04|                   [1005161]|47 seconds|          2|  211.92|          1|
+---------+-------------------

In [18]:
sdf_customer_profile = sdf_session_agg.groupBy("user_id").agg(f.sum("sum(events)").alias("sum_events"), f.sum("sum(views)").alias("sum_views"), f.sum("sum(purchases)").alias("sum_purchases"), f.sum("sum(carts)").alias("sum_carts"), f.sum("turnover").alias("sum_turnover"), f.count("user_session").alias("count_session"), f.sum("successfull").alias("sum_successfull"), f.collect_list("collect_list(bought_product)").alias("bought_product"))

sdf_customer_profile = sdf_customer_profile.withColumn("avg_turnover_per_session", (sdf_customer_profile["sum_turnover"] / sdf_customer_profile["count_session"]))
sdf_customer_profile = sdf_customer_profile.withColumn("avg_events_per_session", (sdf_customer_profile["sum_events"] / sdf_customer_profile["count_session"]))


sdf_customer_profile.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- sum_events: long (nullable = true)
 |-- sum_views: long (nullable = true)
 |-- sum_purchases: long (nullable = true)
 |-- sum_carts: long (nullable = true)
 |-- sum_turnover: double (nullable = true)
 |-- count_session: long (nullable = false)
 |-- sum_successfull: long (nullable = true)
 |-- bought_product: array (nullable = false)
 |    |-- element: array (containsNull = false)
 |    |    |-- element: string (containsNull = false)
 |-- avg_turnover_per_session: double (nullable = true)
 |-- avg_events_per_session: double (nullable = true)



In [19]:
sdf_customer_profile.where(sdf_customer_profile["avg_turnover_per_session"] > 0).show()

+---------+----------+---------+-------------+---------+------------+-------------+---------------+--------------+------------------------+----------------------+
|  user_id|sum_events|sum_views|sum_purchases|sum_carts|sum_turnover|count_session|sum_successfull|bought_product|avg_turnover_per_session|avg_events_per_session|
+---------+----------+---------+-------------+---------+------------+-------------+---------------+--------------+------------------------+----------------------+
|559368633|         3|        2|            1|        0|       566.3|            1|              1|  [[13200605]]|                   566.3|                   3.0|
|513351129|         2|        1|            1|        0|      211.92|            1|              1|   [[1005161]]|                  211.92|                   2.0|
+---------+----------+---------+-------------+---------+------------+-------------+---------------+--------------+------------------------+----------------------+



### Data Export

In [23]:
sdf_customer_profile = sdf_customer_profile.withColumn("bought_product", sdf_customer_profile["bought_product"].cast(pyspark.sql.types.StringType()))

sdf_customer_profile.write.csv("data/customer_profile.csv", mode="overwrite")

Py4JJavaError: An error occurred while calling o486.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at or

## Data Split

In [13]:
rdd = sdf.rdd

In [14]:
(trainingData, testData) = sdf.rdd.randomSplit([0.7, 0.3], seed=123)

In [15]:
rdd.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 605) (DESKTOP-JKUQRDK executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apac

## Modelling

Task:
Cluster the customer in groups:

- high 
- medium
- low
- NULL with potential
- NULL without potential


Target Feature -> Turnover

Next Steps: Find similiarites inside these customer groups:

- Buys same products
- similar behavior -> same marketing

### K-Means

In [14]:
# model
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 605) (DESKTOP-JKUQRDK executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.

In [14]:
# train

In [15]:
# eval

In [1]:

from pyspark import SparkContext as sc
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
# predictions = model.predict(testData.map(lambda x: x.features))
# labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
# testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
# print('Test Error = ' + str(testErr))
# print('Learned classification forest model:')
# print(model.toDebugString())



AttributeError: 'str' object has no attribute 'defaultParallelism'

### Random Forrest

### K-Means

### DBSCAN

## General Comparison

## Hyperparamter & extended Stuff